# ANEEL Public API Data

## Importing libraries

In [7]:
import requests
import polars as pl
from datetime import datetime as time

## Defining a class
Just to deal with the data

In [3]:
class DataHandler:
    def __init__(self):
        self.base_url = "https://dadosabertos.aneel.gov.br"
        self.result_total = self.get_result_total()

    
    def get_result_total(self) -> str:
        url = self.get_initial_url(0)
        answer = requests.get(url).json()
        return answer.get("result").get("total")

    
    def get_initial_url(self, limit: int) -> str:
        return f"{self.base_url}/api/3/action/datastore_search" + \
            f"?resource_id=b1bd71e7-d0ad-4214-9053-cbd58e9564a7&limit={limit}"

    
    def load_data(self) -> pl.LazyFrame:        
        url = self.get_initial_url(self.result_total)
        answer = requests.get(url).json()
        records = answer.get("result").get("records")
        lf = pl.LazyFrame(records)
        
        return lf.with_columns(pl.col("MdaPotenciaInstaladaKW").str.replace(",", ".")
            .cast(pl.Float64)
        )


    
    # Currently not being needed.
    # In case pagination become attractive just use it.
    def load_data_batch(self,
        batch_size: int = 100,
        url: str = "",
        request_offset: int = 0,
        lazyframes: [pl.LazyFrame] = []
    ) -> pl.LazyFrame:


        init_time = datetime.datetime.now()
        if not url:
            url = self.get_initial_url(self.request_limit)
        
        
        number_of_records = request_offset + batch_size
        print(f"Loaded: {number_of_records}/{answer.get("result").get("total")}")

        
        answer = requests.get(url).json()
        
        records = answer.get("result").get("records")
        records = [f"{record}\n" for record in records]
        lazyframes.append(
            pl.scan_ndjson(records)
        )


        if (
            number_of_records >=
            answer.get("result").get("total")
        ):
            print(f"Time: {datetime.datetime.now() - init_time}")

            return pl.concat(lazyframes)

        else:            
            url = self.base_url + answer.get("result").get("_links").get("next")
            request_offset += self.request_limit


            print(f"Time: {datetime.datetime.now() - init_time}")
            self.load_data(url, request_offset, lazyframes)

## Putting it into work

In [3]:
initial_time = time.now()

dd = DataHandler()
first_request_time = time.now()

lf = dd.load_data()
final_time = time.now()

print(f"First request: {first_request_time - initial_time};\n" +
        f"Second request: {final_time - first_request_time};\n" +
        f"Total: {final_time - initial_time}")

First request: 0:00:00.796949;
Second request: 0:00:11.997531;
Total: 0:00:12.794480


## Code for manualy testing the API

In [8]:
BASE_URL = "https://dadosabertos.aneel.gov.br/api/3/action/datastore_search"

params = {
    "resource_id": 'b1bd71e7-d0ad-4214-9053-cbd58e9564a7',
    "limit": 32000,
    "offset": 3520000,
}
r = requests.get(BASE_URL, params=params).json()

In [9]:
r#.get("result").get("records")[0])#.get("next")#.json()#.get("result").get("total")

{'help': 'https://dadosabertos.aneel.gov.br/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 32000,
  'offset': 3520000,
  'records_format': 'objects',
  'resource_id': 'b1bd71e7-d0ad-4214-9053-cbd58e9564a7',
  'total_estimation_threshold': None,
  'records': [{'_id': 3520001,
    'DatGeracaoConjuntoDados': '2025-06-22',
    'AnmPeriodoReferencia': '06/2025',
    'NumCNPJDistribuidora': '33050196000188',
    'SigAgente': 'CPFL-PAULISTA',
    'NomAgente': 'COMPANHIA PAULISTA DE FORCA E LUZ',
    'CodClasseConsumo': '3',
    'DscClasseConsumo': 'Rural',
    'CodSubGrupoTarifario': '10',
    'DscSubGrupoTarifario': 'B2',
    'CodUFibge': '35',
    'SigUF': 'SP',
    'CodRegiao': '3504',
    'NomRegiao': 'Sudeste',
    'CodMunicipioIbge': '3506003',
    'NomMunicipio': 'Bauru',
    'CodCEP': '17110000',
    'SigTipoConsumidor': 'PJ',
    'NumCPFCNPJ': '08834452000180',
    'NomTitularEmpreendimento': 'JOSE CARLOS VICARI',
    'C

In [2]:
from os.path import exists

import polars as pl

from time import time

class DataHandler:
    def __init__(self):
        self.DATA_PATH = "./data/"
        self.DATA_FILE = "aneel_data.parquet"
        self.DATA_FILE_PATH = self.DATA_PATH + self.DATA_FILE
        self.data_loader = DataLoader(self.DATA_FILE_PATH)

    def LazyFrame(self):
        has_no_data_file = not exists(self.DATA_FILE_PATH)

        if has_no_data_file:
            ti = time()
            self.data_loader.load()
            print(f"Loading data took {time() - ti:.2f} seconds.")

        lf = pl.scan_parquet(self.DATA_FILE_PATH)

        return lf.with_columns(
            pl.col("MdaPotenciaInstaladaKW").str.replace(",", ".")
            .cast(pl.Float64)
        )

In [3]:
from concurrent.futures import ThreadPoolExecutor
from threading import Lock, Event

import pyarrow.parquet as pq
import pyarrow as pa

from time import sleep
from queue import PriorityQueue

import requests
from requests import ConnectionError

class DataLoader:
    def __init__(self, data_file_path: str):
        self.BASE_URL = "https://dadosabertos.aneel.gov.br/api/3/action/datastore_search"
        self.RESOURCE_ID = "b1bd71e7-d0ad-4214-9053-cbd58e9564a7"
        self.DATA_FILE_PATH = data_file_path
        self.REQUESTS_LIMIT = 32000
        self.lock = Lock()
        self.to_write = PriorityQueue()


    def load(self) -> None:
        try:
            answer = self.make_request(0)
            total, records = self.get_request_infos(answer)
            self.to_write.put((0, records))

            table = pa.Table.from_pylist(records)
            number_of_threads = total // self.REQUESTS_LIMIT

            with ThreadPoolExecutor(max_workers=31) as executor:
                writer_future = executor.submit(self.write_loads, table.schema, number_of_threads + 1)
                offset = self.REQUESTS_LIMIT
                for i in range(number_of_threads):
                    executor.submit(self.load_request, offset)
                    offset += self.REQUESTS_LIMIT
                    sleep(0.1)
                print("out", number_of_threads, total)
                writer_future.result()
        except Exception as e:
            raise ConnectionError(f"Some error: {e}")
        finally:
            print("DataLoader finished execution.")


    def make_request(self, offset: int)-> str:
        try:
            print(f"*\tMaking Request, offset: {offset}.")
            params = {
                "resource_id": self.RESOURCE_ID,
                "limit": self.REQUESTS_LIMIT,
                "offset": offset
            }

            answer = requests.get(self.BASE_URL, params=params).json()
            if not answer.get("success"):
                raise ConnectionError("Aneel service signalizing unsuccessful answer!")

            print(f"**\tDone Request, offset: {offset}.")
            return answer.get("result")
        except:
            raise ConnectionError("Some error making a request")


    def load_request(self, offset: int) -> None:
        try:
            answer = self.make_request(offset)
            _, records = self.get_request_infos(answer)
            self.to_write.put((offset, records))
        except Exception as e:
            print(f"Error in load_request: {e}")


    def get_request_infos(self, answer: dict)-> tuple[int, list[dict]]:
        total = answer.get("total")
        records = answer.get("records")
        return (total, records)

    def write_loads(self, schema: pa.Schema, num_producers: int) -> None:
        finished = 0
        total = 0
        with pq.ParquetWriter(self.DATA_FILE_PATH, schema) as writer:
            zero = 0
            while finished < num_producers:
                _, records = self.to_write.get()
                total += len(records) if records is not None else 0
                if records:
                    zero += 32000
                    print(f"zero {zero}, rec: {records[-1]['_id']}")
                    finished += 1
                else:
                    print("No more records to write.")
                    continue
                print("finished", finished, self.to_write.qsize())
                print(total) if finished < 105 else print(total, records[0], records[-1])
                    
                table = pa.Table.from_pylist(records)
                writer.write_table(table)
        print("Closing writer")

In [4]:
dh = DataHandler()
lf = dh.LazyFrame()
lf.collect()

*	Making Request, offset: 0.
**	Done Request, offset: 0.
*	Making Request, offset: 32000.
zero 32000, rec: 32000
finished 1 0
32000
*	Making Request, offset: 64000.
*	Making Request, offset: 96000.
*	Making Request, offset: 128000.
*	Making Request, offset: 160000.
*	Making Request, offset: 192000.
*	Making Request, offset: 224000.
*	Making Request, offset: 256000.
*	Making Request, offset: 288000.
*	Making Request, offset: 320000.
*	Making Request, offset: 352000.
*	Making Request, offset: 384000.
*	Making Request, offset: 416000.
*	Making Request, offset: 448000.
*	Making Request, offset: 480000.
*	Making Request, offset: 512000.
*	Making Request, offset: 544000.
*	Making Request, offset: 576000.
*	Making Request, offset: 608000.
*	Making Request, offset: 640000.
*	Making Request, offset: 672000.
*	Making Request, offset: 704000.
*	Making Request, offset: 736000.
*	Making Request, offset: 768000.
*	Making Request, offset: 800000.
*	Making Request, offset: 832000.
*	Making Request, of

KeyboardInterrupt: 

In [1]:
from os.path import exists

import polars as pl

import requests
from requests import ConnectionError

class DataHandler:
    def __init__(self):
        self.DATA_PATH = "./data/"
        self.DATA_FILE = "aneel_threading.parquet"
        self.DATA_FILE_PATH = self.DATA_PATH + self.DATA_FILE
        self.data_loader = DataLoader(self.DATA_FILE_PATH)

    def LazyFrame(self):
        has_no_data_file = not exists(self.DATA_FILE_PATH)

        if has_no_data_file:
            self.data_loader.load()

        lf = pl.scan_parquet(self.DATA_FILE_PATH)

        return lf.with_columns(
            pl.col("MdaPotenciaInstaladaKW").str.replace(",", ".")
            .cast(pl.Float64)
        )

In [2]:
from concurrent.futures import ThreadPoolExecutor
from threading import Lock, Event

import pyarrow.parquet as pq
import pyarrow as pa
import polars as pl

from queue import Queue

import requests
from requests import ConnectionError

class DataLoader:
    def __init__(self, data_file_path: str):
        self.BASE_URL = "https://dadosabertos.aneel.gov.br/api/3/action/datastore_search"
        self.RESOURCE_ID = "b1bd71e7-d0ad-4214-9053-cbd58e9564a7"
        self.DATA_FILE_PATH = data_file_path
        self.REQUESTS_LIMIT = 32000
        self.lock = Lock()
        self.to_write = Queue()


    def load(self)-> None:      
        try:
            answer = self.make_request(0)
            total, records = self.get_request_infos(answer)
            self.to_write.put(records)

            table = pa.Table.from_pylist(records)

            number_of_threads = (total - 1) // self.REQUESTS_LIMIT
            threading_events = [Event().set()] + [Event() for _ in range(number_of_threads)]
            with ThreadPoolExecutor(max_workers=10) as executor:
                executor.submit(self.write_loads, threading_events, table.schema)

                offset = self.REQUESTS_LIMIT
                for i, current_thread in enumerate(threading_events[1:]):
                    previous_thread = threading_events[i-1]
                    execution_params = [previous_thread, current_thread, offset]
                    
                    executor.submit(self.load_request, *execution_params)
                    offset += self.REQUESTS_LIMIT
        except:
            raise ConnectionError("Some error")


    def make_request(self, offset: int)-> str:
        try:
            print(f"*\tMaking Request, offset: {offset}.")
            params = {
                "resource_id": self.RESOURCE_ID,
                "limit": self.REQUESTS_LIMIT,
                "offset": offset
            }

            answer = requests.get(self.BASE_URL, params=params).json()
            if not answer.get("success"):
                raise ConnectionError("Aneel service signalizing unsuccessful answer!")

            print(f"**\tDone Request, offset: {offset}.")
            return answer.get("result")
        except:
            raise ConnectionError("Some error making a request")


    def load_request(
        self, 
        previous_thread: None|Event, 
        current_thread: Event, 
        offset: int, 
        writer: pq.ParquetWriter
    )-> None:

        try:
            print(f"> Starting Thread, offset: {offset}.")
            answer = self.make_request(offset)
            total, records = self.get_request_infos(answer)

            print(f"[ Locking Thread, offset: {offset}. Waiting {previous_tread}")
            previous_thread.wait()
            self.to_write.put(records)
            current_thread.set()
            print(f"] Unocking Thread, offset: {offset}. Waiting {previous_tread}")
            print(f"=> Loading data, {100*offset/total:.1f}%; {offset} of {total}.")
        except:
            raise ConnectionError("A loading error")



    def get_request_infos(self, answer: dict)-> tuple[int, list[dict]]:
        total = answer.get("total")
        records = answer.get("records")
        return (total, records)

    def write_loads(self, events: list[Event], schema: pa.Schema)-> None:
        print("Writer Thread")
        try:
            print("ON")
            def needs_writing()-> bool:
                if not self.to_write.empty():
                    return True

                for event in events:
                    if not event.is_set():
                        return True

                return False

            print("After, ", needs_writing())
            with pq.ParquetWriter(self.DATA_FILE_PATH, schema) as writer:
                c = 1
                while need_writing():
                    print("IN")
                    if self.to_write:
                        records = self.to_write.get()
                        c += 1
                        print("New writing, ", c)
                        table = pa.Table.from_pylist(records)
                        writer.write_table(table)
        except:
            raise ConnectionError("A Writing error")


In [5]:
from aneel_backend.DataHandler import DataHandler
from aneel_backend.DataLoader import DataLoader

dh = DataHandler()
lf = dh.LazyFrame()

In [4]:
lf.collect()

_id,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,CodUFibge,SigUF,CodRegiao,NomRegiao,CodMunicipioIbge,NomMunicipio,CodCEP,SigTipoConsumidor,NumCPFCNPJ,NomTitularEmpreendimento,CodEmpreendimento,DthAtualizaCadastralEmpreend,SigModalidadeEmpreendimento,DscModalidadeHabilitado,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,NumCoordNEmpreendimento,NumCoordEEmpreendimento,MdaPotenciaInstaladaKW,NomSubEstacao,NumCoordESub,NumCoordNSub
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str
1,"""2025-06-22""","""06/2025""","""08336783000190""","""CELESC-DIS""","""CELESC DISTRIBUICAO S.A""","""3""","""Rural""","""10""","""B2""","""42""","""SC""","""4202""","""Sul""","""4204509""","""Corupá""","""89278***""","""PF""","""***.371.499-**""","""***""","""""","""2024-02-01""","""P""","""Com Microgeracao ou Minigeraca…","""1""","""UFV""","""Radiação solar""","""Microgeracao""","""-5,06""","""-42,78""",6.0,"""""","""""",""""""
2,"""2025-06-22""","""06/2025""","""08336783000190""","""CELESC-DIS""","""CELESC DISTRIBUICAO S.A""","""1""","""Residencial""","""9""","""B1""","""42""","""SC""","""4202""","""Sul""","""4217402""","""Schroeder""","""89275***""","""PF""","""***.744.159-**""","""***""","""""","""2024-02-01""","""P""","""Com Microgeracao ou Minigeraca…","""1""","""UFV""","""Radiação solar""","""Microgeracao""","""-5,06""","""-42,78""",5.0,"""""","""""",""""""
3,"""2025-06-22""","""06/2025""","""08336783000190""","""CELESC-DIS""","""CELESC DISTRIBUICAO S.A""","""3""","""Rural""","""10""","""B2""","""42""","""SC""","""4201""","""Sul""","""4214151""","""Princesa""","""89935***""","""PF""","""***.249.519-**""","""***""","""""","""2024-02-01""","""P""","""Com Microgeracao ou Minigeraca…","""1""","""UFV""","""Radiação solar""","""Microgeracao""","""-5,06""","""-42,78""",3.0,"""""","""""",""""""
4,"""2025-06-22""","""06/2025""","""08336783000190""","""CELESC-DIS""","""CELESC DISTRIBUICAO S.A""","""1""","""Residencial""","""9""","""B1""","""42""","""SC""","""4201""","""Sul""","""4216701""","""São José do Cedro""","""89930***""","""PF""","""***.938.279-**""","""***""","""""","""2024-02-01""","""P""","""Com Microgeracao ou Minigeraca…","""1""","""UFV""","""Radiação solar""","""Microgeracao""","""-5,06""","""-42,78""",5.0,"""""","""""",""""""
5,"""2025-06-22""","""06/2025""","""08336783000190""","""CELESC-DIS""","""CELESC DISTRIBUICAO S.A""","""1""","""Residencial""","""9""","""B1""","""42""","""SC""","""4201""","""Sul""","""4216255""","""São João do Oeste""","""89897***""","""PF""","""***.178.400-**""","""***""","""""","""2024-02-01""","""P""","""Com Microgeracao ou Minigeraca…","""1""","""UFV""","""Radiação solar""","""Microgeracao""","""-5,06""","""-42,78""",6.0,"""""","""""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3603908,"""2025-06-22""","""06/2025""","""25086034000171""","""ETO""","""ENERGISA TOCANTINS DISTRIBUIDO…","""1""","""Residencial""","""9""","""B1""","""17""","""TO""","""1701""","""Norte""","""1711902""","""Lagoa da Confusão""","""77493***""","""PF""","""***.849.198-**""","""***""","""GD.TO.003.324.328""","""2025-06-13""","""P""","""Com Microgeracao ou Minigeraca…","""1""","""UFV""","""Radiação solar""","""Microgeracao""","""""","""""",6.0,"""""","""""",""""""
3603909,"""2025-06-22""","""06/2025""","""25086034000171""","""ETO""","""ENERGISA TOCANTINS DISTRIBUIDO…","""1""","""Residencial""","""9""","""B1""","""17""","""TO""","""1701""","""Norte""","""1709500""","""Gurupi""","""77430***""","""PF""","""***.624.301-**""","""***""","""GD.TO.003.324.330""","""2025-06-12""","""P""","""Com Microgeracao ou Minigeraca…","""1""","""UFV""","""Radiação solar""","""Microgeracao""","""-11,72""","""-49,10""",6.0,"""""","""""",""""""
3603910,"""2025-06-22""","""06/2025""","""25086034000171""","""ETO""","""ENERGISA